In [1]:
## Import necessary packages
%matplotlib inline
import os
import csv
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import linregress

In [2]:
# Load in the Austin crime dataset
# df = os.path.join('Resources', 'Crime_Reports.csv')

datafile = '../Resources/Crime_Reports.csv'

In [3]:
data = pd.read_csv(datafile)
data.head(100)

,Incident Number,Highest Offense Description,Highest Offense Code,Family Violence,Occurred Date Time,Occurred Date,Occurred Time,Report Date Time,Report Date,Report Time,...,Census Tract,Clearance Status,Clearance Date,UCR Category,Category Description,X-coordinate,Y-coordinate,Latitude,Longitude,Location
0,2.017115e+08,AGG ASLT STRANGLE/SUFFOCATE,410,Y,1/1/2017 10:30,1/1/2017,1030,1/1/2017 11:58,1/1/2017,1158,...,NaN,C,1/10/2017,13A,Aggravated Assault,3130704.0,3130704.0,30.359794,-97.688358,"(30.35979391, -97.688358)"
1,2.017100e+08,AGG ASSAULT,402,N,1/1/2017 0:07,1/1/2017,7,1/1/2017 0:07,1/1/2017,7,...,NaN,NaN,NaN,13A,Aggravated Assault,3114768.0,3114768.0,30.268162,-97.741362,"(30.26816195, -97.74136237)"
2,2.017109e+08,AGG ASSAULT,402,N,1/1/2017 4:20,1/1/2017,420,1/1/2017 4:20,1/1/2017,420,...,NaN,N,3/8/2017,13A,Aggravated Assault,3131193.0,3131193.0,30.318604,-97.687937,"(30.3186038, -97.68793685)"
3,2.017503e+10,AGG ASSAULT FAM/DATE VIOLENCE,402,Y,1/1/2017 0:00,1/1/2017,0,8/10/2017 8:30,8/10/2017,830,...,NaN,O,8/17/2017,13A,Aggravated Assault,3099695.0,3099695.0,30.196029,-97.791021,"(30.1960291, -97.79102086)"
4,2.019239e+10,AGG FORCED SODOMY,1718,N,1/1/2017 21:29,1/1/2017,2129,8/27/2019 21:29,8/27/2019,2129,...,NaN,C,10/29/2019,11B,Rape,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2.017108e+08,DWI .15 BAC OR ABOVE,2111,N,1/1/2017 3:19,1/1/2017,319,1/1/2017 3:19,1/1/2017,319,...,NaN,C,1/14/2017,NaN,NaN,3122483.0,3122483.0,30.304849,-97.715922,"(30.30484886, -97.71592248)"
96,2.017125e+08,DWI 2ND,2102,N,1/1/2017 22:10,1/1/2017,2210,1/1/2017 22:10,1/1/2017,2210,...,NaN,C,1/2/2017,NaN,NaN,3120610.0,3120610.0,30.398318,-97.719328,"(30.3983179, -97.71932765)"
97,2.017124e+08,DWI 2ND,2102,N,1/1/2017 21:11,1/1/2017,2111,1/1/2017 21:11,1/1/2017,2111,...,NaN,C,1/2/2017,NaN,NaN,3115476.0,3115476.0,30.193997,-97.741110,"(30.19399698, -97.74111045)"
98,2.017119e+08,EVADING / FOOT,2723,N,1/1/2017 16:19,1/1/2017,1619,1/1/2017 16:19,1/1/2017,1619,...,NaN,C,1/1/2017,NaN,NaN,3125100.0,3125100.0,30.261960,-97.708794,"(30.26195977, -97.70879365)"


In [4]:
data.columns

Index(['Incident Number', 'Highest Offense Description',
       'Highest Offense Code', 'Family Violence', 'Occurred Date Time',
       'Occurred Date', 'Occurred Time', 'Report Date Time', 'Report Date',
       'Report Time', 'Location Type', 'Address', 'Zip Code',
       'Council District', 'APD Sector', 'APD District', 'PRA', 'Census Tract',
       'Clearance Status', 'Clearance Date', 'UCR Category',
       'Category Description', 'X-coordinate', 'Y-coordinate', 'Latitude',
       'Longitude', 'Location'],
      dtype='object')

In [5]:

Newdata = data.drop(columns=["Incident Number", "Highest Offense Code", "Family Violence", 'Occurred Date', 'Report Date Time', 'Report Date',
       'Report Time', 'Address', 'Council District', 'APD Sector', 'Clearance Status', 'Clearance Date', 'UCR Category', 'X-coordinate', 'Y-coordinate', 'Latitude', 'Longitude'])
Newdata.head()

,Highest Offense Description,Occurred Date Time,Occurred Time,Location Type,Zip Code,APD District,PRA,Census Tract,Category Description,Location
0,AGG ASLT STRANGLE/SUFFOCATE,1/1/2017 10:30,1030,HOTEL / MOTEL / ETC.,78753.0,1,240.0,NaN,Aggravated Assault,"(30.35979391, -97.688358)"
1,AGG ASSAULT,1/1/2017 0:07,7,STREETS / HWY / ROAD / ALLEY,78701.0,2,381.0,NaN,Aggravated Assault,"(30.26816195, -97.74136237)"
2,AGG ASSAULT,1/1/2017 4:20,420,PARKING LOTS / GARAGE,78723.0,3,294.0,NaN,Aggravated Assault,"(30.3186038, -97.68793685)"
3,AGG ASSAULT FAM/DATE VIOLENCE,1/1/2017 0:00,0,RESIDENCE / HOME,78745.0,3,526.0,NaN,Aggravated Assault,"(30.1960291, -97.79102086)"
4,AGG FORCED SODOMY,1/1/2017 21:29,2129,RESIDENCE / HOME,78753.0,7,241.0,NaN,Rape,NaN
